# MIRAC-5 Reduce Helper

This jupyter notebook is intended to both provide instructions on how to use the mirac5reduce package and serve as a simple user interface for performing their own simple reductions.

To use this notebook, always start with the first two sections:

1. [Set Up the Config File](#setup-config)

2. [Set Up Python Environment](#setup-pyenv)

The remaining sections can be accessed as needed to perform various tasks:

3. [Calculate Mean Dark File](#calc-meandark)

4. [Create Bad Pixel Mask](#calc-bpmask)

5. [Create Flatfield File](#calc-flatfield)

6. [Create Mean Flat File](#calc-meanflat)

7. [Reduce Chop/Nod Observations](#reduce-chopnod)

8. [Reduce Staring Mode Observation](#reduce-staring)


<a id="setup-config"></a>
## Set Up the Config File

First, you will want to create your own copy of the config file and edit it. 

The sample config file is provided as part of the mirac5reduce package under `docs/runparams.init`. 

Create a copy of that file, preferably somewhere near where your reduction files will be, though that isn't required. 

Enter the path and file name of this copy in the following cell to save it as the variable, `config_file`:

In [ ]:
config_file = ''

Open your config file in your text editor of choice. You will see that it has 4 sections: `[REDUCTION]`, `[CALIB]`, `[COMPUTING]`, and `[DATA_ARCH]`.

The last two sections, `[COMPUTING]` and `[DATA_ARCH]` contain parameters that are specific to the computer that you are running the reduction on and the architecture of the data produced by the MIRAC-5 data aqcuisition software, respectively. 

**Review and set the parameters in the `[COMPUTING]` and `[DATA_ARCH]` sections of your config file.** They should not change for the rest of the reduction. Descriptions are provided in the template config file. 

The remaining sections, `[REDUCTION]` and `[CALIB]` contain parameters that can either be provided directly when calling a function or imported from this config file to simplify function calls.

To aid in flexibility, this notebook will allow most of the parameters to be provided directly on function calls. But we do assume that (1) all raw data is stored in the same path and (2) all raw calibration files are stored in the same path. (These paths may be the same or different.)

**Review and set the following parameters in your config file:**
* `raw_data_path` in the `[REDUCTION]` section
* `raw_cals_path` in the `[CALIB]` section

**Make sure all other keyword values in these two sections are empty and save your config file.**


<a id="setup-pyenv"></a>
## Set Up Python Environment

Briefly, we import the scripts that contain the main upper-level functions:

In [ ]:
from mirac5reduce.reduce import combine_frames
from mirac5reduce.cal import bpmask, flatfield

In addition, all of the upper-level functions in `mirac5reduce` have the option to write output to a log file instead of printing it to the terminal. Use the cell below to set your log file or update it if you decide to start a new one:

In [ ]:
# To write output to a log file, uncomment the line below and fill in the path and file name for the log file
# logfile = ''
# Or, to just write output directly to terminal, uncomment the following line instead
# logfile = None

Finally, import a few side-packages that will help with visualization in this notebook:

In [ ]:
from astropy.io import fits
from matplotlib import pyplot as P
import numpy as np
from mirac5reduce.utils.statfunc import medabsdev
import configparser, os

<a id="calc-meandark"></a>
## Calculate Mean Dark File

The function, `combine_frames.meanframe` calculates the mean frame for a range of file numbers and saves the output to a fits file with a populated header. While this function is general, it determines which raw file path to use from the config file from the data type specified when calling it.

In [ ]:
# Enter the start and end file numbers (inclusive) of the range of raw dark files that you want to average
raw_dark_startno = 
raw_dark_endno   = 

# Enter the (FITS) file name of the mean dark file that you want to create. 
#     It will be saved in your calib_outpath.
mean_dark_filename = 'dark_{0}_{1}.fits'.format( raw_dark_startno, raw_dark_endno )

# Then calculate the mean dark frame. Turns on debugging mode to echo the parameters provided.
combine_frames.meanframe( config_file, 'dark', logfile = logfile, debug = True,
                          startno = raw_dark_startno, endno = raw_dark_endno, outfile = mean_dark_filename )

Use this cell to preview this file and its header:

In [ ]:
## Generates file preview only; no editing needed ##
conf = configparser.ConfigParser()
_ = conf.read(config_file)
calib_outpath = conf['CALIB']['calib_outpath']
with fits.open( os.path.join(calib_outpath,mean_dark_filename) ) as hdulist:
    
    print('Mean Value: {0:.2e} DN'.format( np.mean(hdulist[0].data) ))
    print('Median Value: {0:.2e} DN'.format( np.median(hdulist[0].data) ))
    print('Std Dev: {0:.2e} DN'.format( np.std(hdulist[0].data) ))
    print('Med. Abs. Dev.: {0:.2e} DN'.format( medabsdev(hdulist[0].data) ))
    
    fig, axes = P.subplots(figsize = (11,5),  ncols=2)
    axes[0].imshow( hdulist[0].data )
    axes[1].set_yscale('log')
    axes[1].hist(hdulist[0].data.ravel(), bins=500)

<a id="calc-bpmask"></a>
## Create Bad Pixel Mask

The bad pixel mask is created with a simple sigma-filter on the median absolute deviation of the mean dark file created above.

In [ ]:
# Uses the same mean_dark_filename as above

# The threshold above and below which pixels are considered bad is 
#     this number x the M.A.D. on either side of the median
bp_threshold = 7.0

# The path and (FITS) file name to which the bad pixel mask will be saved
bp_filename = 'bpmask_{0}_{1}.fits'.format( raw_dark_startno, raw_dark_endno )

# Create the bad pixel mask file. Turns on debugging mode to echo the parameters provided.
bpmask.make_bpmask( config_file, logfile = logfile, debug = True,
                    dark_file = mean_dark_filename, bp_threshold = bp_threshold, outfile = bp_filename )

While the histogram from the mean dark file above may provide more insight, the cell below can be used for a basic summary of the mask data.

In [ ]:
## Run cell for file preview only; no editing needed ##
conf = configparser.ConfigParser()
_ = conf.read(config_file)
calib_outpath = conf['CALIB']['calib_outpath']
with fits.open( os.path.join(calib_outpath,bp_filename) ) as hdulist:
    
    print('Flagged {0} / {1} pixels ({2:.1f}%)'.format( np.sum(hdulist[0].data), hdulist[0].data.size, 
                                                       100. * np.sum(hdulist[0].data) / hdulist[0].data.size ))
    print('\n{0} ext 0 header:\n'.format(bp_filename))
    for key in hdulist[0].header.keys():
        print( '{0: <8} = {1: >24} / {2}'.format( *hdulist[0].header.cards[key] ) )
    

<a id="calc-flatfield"></a>
## Create Flatfield File

To calculate a dark subtracted and scaled mean flatfield frame, we need to use the `flatfield.create_flatfield` function:

In [ ]:
# Enter the start and end file numbers (inclusive) of the range of raw flat files to use
raw_flat_startno = 
raw_flat_endno   = 

# Enter the (FITS) file name of the flatfield file that you want to create. Saved in your calib_outpath.
flatfield_filename = 'flatfield_{0}_{1}.fits'.format( raw_flat_startno, raw_flat_endno )

# Enter the name of the *existing* mean dark file to subtract off of the flats
mean_dark_filename = 'dark_{0}_{1}.fits'.format( raw_dark_startno, raw_dark_endno )

# If you wish to mask bad pixels from the flats before they are normalized, enter the name of the existing bad 
#     pixel mask to use here. Otherwise, set the following to None:
bp_filename = 'bpmask_{0}_{1}.fits'.format( raw_dark_startno, raw_dark_endno )

# Create the flatfield file. Turns on debugging mode to echo the parameters provided.
flatfield.create_flatfield( config_file, logfile = logfile, debug = True,
                            startno = raw_flat_startno, endno = raw_flat_endno, outfile = flatfield_filename,
                            dark_file = mean_dark_filename, bpmask_file = bp_filename )


<a id="calc-meanflat"></a>
## Create Mean Flat File

Just as with the mean dark file, the `combine_frames.meanframe` function can be used to create a mean flat frame instead of a mean dark frame. Unlike `flatfield.create_flatfield` above, this is a simple mean of the frames provided, with no dark subtraction or scaling.

In [ ]:
# Enter the start and end file numbers (inclusive) of the range of raw flat files that you want to average
raw_flat_startno = 
raw_flat_endno   = 

# Enter the path and (FITS) file name of the mean flat fits file that you want to create
mean_flat_filename = 'flat_{0}_{1}.fits'.format( raw_flat_startno, raw_flat_endno )

# Then calculate the mean flat frame. Turns on debugging mode to echo the parameters provided.
combine_frames.meanframe( config_file, 'flat', logfile = logfile, debug = True,
                          startno = raw_flat_startno, endno = raw_flat_endno, outfile = mean_flat_filename )

Again, the cell below just offers a preview of the contents of the created file:

In [ ]:
## Run for file preview only; no editing needed ##
conf = configparser.ConfigParser()
_ = conf.read(config_file)
calib_outpath = conf['CALIB']['calib_outpath']
with fits.open( os.path.join(calib_outpath,mean_flat_filename) ) as hdulist:
    
    print('Mean Value: {0:.2e} DN'.format( np.mean(hdulist[0].data) ))
    print('Median Value: {0:.2e} DN'.format( np.median(hdulist[0].data) ))
    print('Std Dev: {0:.2e} DN'.format( np.std(hdulist[0].data) ))
    print('Med. Abs. Dev.: {0:.2e} DN'.format( medabsdev(hdulist[0].data) ))
    
    fig, axes = P.subplots(figsize = (11,4),  ncols=2)
    axes[0].imshow( hdulist[0].data )
    axes[1].set_yscale('log')
    axes[1].hist(hdulist[0].data.ravel(), bins=500)

<a id="reduce-chopnod"></a>
## Reduce Chop/Nod Observations

Reduction of the chop nod data is currently limited to calculating the mean difference frame of all of the chop/nodded frames with `combine_frames.chopnodframe`.

It can be ran with the following:

In [ ]:
# Enter the start and end file numbers (inclusive) of the range of raw chop/nod files that you want to average
raw_data_startno = 
raw_data_endno   = 

# Set the chop and nod frequencies, in Hz
chopfreq = 
nodfreq  = 

# Enter the path and (FITS) file name of the resulting mean chop/nod diff fits file that you want to create
chopnod_filename = 'chopnod_{0}_{1}.fits'.format( raw_data_startno, raw_data_endno )

# Then calculate and save the mean difference frame. Turns on debugging mode to echo the parameters provided.
combine_frames.chopnodframe( config_file, logfile = logfile, debug = True,
                             startno = raw_data_startno, endno = raw_data_endno,
                             chopfreq = chopfreq, nodfreq = nodfreq, outfile = chopnod_filename )

The cell below provides a preview, with optional masking:

In [ ]:
# Set bool specifying whether you want to apply bad pixel mask created above to the plotted figures
plot_with_mask = False



## Run for image preview only; no editing needed ##
conf = configparser.ConfigParser()
_ = conf.read(config_file)
reduce_outpath = conf['REDUCTION']['reduce_outpath']
cn_data = fits.getdata( os.path.join(reduce_outpath,chopnod_filename), 0 )
if plot_with_mask:
    mask = fits.getdata( bp_filename, 0 )
    cn_data = np.ma.masked_array( cn_data, mask=mask )
    cn_data = cn_data.filled( np.nan )
fig, ax = P.subplots(figsize = (9,7))
im = ax.imshow( cn_data, origin='lower' )
fig.colorbar(im, ax=ax)


# Use this section to specify the x/y axis limits on the figure
# xlim = [ 462, 562 ]
# ylim = [ 462, 562 ]
# ax.set_xlim( xlim[0], xlim[1] )
# ax.set_ylim( ylim[0], ylim[1] )

If you know the chop/nod throw, in pixels, you can recombine the image here:

In [ ]:
# Assumes parallel chop/nod throws, where chop-1-nod-A and chop-2-nod-B are at the same location. 
# Specify the throw of one of the two negative chop/nod positions with respect 1A/2B position as [dx, dy], in pix, 
#    where a positive dx value indicates that the negative source imprint is to the right of the positive source
#    The other negative imprint will be assumed to be the opposite direction from the positive central source
offset = [ , ]

# combines frames, but does *not* trim around the positive source
frame_dy = cn_data.shape[0] - abs( 2*offset[1] )
frame_dx = cn_data.shape[1] - abs( 2*offset[0] )
x0 = abs(offset[0]); y0 = abs(offset[1])
recomb_frame =   cn_data[ y0 : frame_dy+y0, x0 : frame_dx+x0 ] - \
                 cn_data[ y0+offset[1] : frame_dy+y0+offset[1], x0+offset[0] : frame_dx+x0+offset[0] ] - \
                 cn_data[ y0-offset[1] : frame_dy+y0-offset[1], x0-offset[0] : frame_dx+x0-offset[0] ]
print('Recombined image array is (y,x) = {0}'.format(recomb_frame.shape))

# Plots recombined frame
fig, ax = P.subplots(figsize = (9,7))
im = ax.imshow( recomb_frame, origin='lower' )
fig.colorbar(im, ax=ax)

# Use this section to zoom into where your source is in the image
# xlim = [ 412, 512 ]
# ylim = [ 457, 557 ]
# ax.set_xlim( xlim[0], xlim[1] )
# ax.set_ylim( ylim[0], ylim[1] )

<a id="reduce-staring"></a>
## Reduce Staring Mode Observations

While not yet implemented in `mirac5reduce` in a fully automated way, staring mode observations can be performed by manually applying the calibration files created above.

First, *create a mean dark file, a bad pixel file, and a flatfield file* using the appropriate sections above. 

Note: It is assumed that all file types have the same integration time!

Then use the following cell to calculate the mean frame of your staring mode data, subtract the mean dark, and divide by the flatfield:

In [ ]:
# Enter the start and end file numbers (inclusive) of the range of raw staring mode files
raw_data_startno = 
raw_data_endno   = 

# Enter the names of your mean dark file, bad pix mask file, and flatfield file. They should all be 
#     saved in your calib_outpath
mean_dark_filename = 'dark_{0}_{1}.fits'.format( raw_dark_startno, raw_dark_endno )
bp_filename        = 'bpmask_{0}_{1}.fits'.format( raw_dark_startno, raw_dark_endno )
flatfield_filename = 'flatfield_{0}_{1}.fits'.format( raw_flat_startno, raw_flat_endno )

# Enter the name of the (FITS) file for the uncorrected mean data frame and the corrected mean data frame, resp.
#     Both will be saved to your reduce_outpath.
mean_rawdata_filename = 'staring_raw_{0}_{1}.fits'.format( raw_data_startno, raw_data_endno )
staring_data_filename = 'staring_reduced_{0}_{1}.fits'.format( raw_data_startno, raw_data_endno )

# Then calculate and save the mean raw frame. Turns on debugging mode to echo the parameters provided.
combine_frames.meanframe( config_file, 'obs', logfile = logfile, debug = True,
                          startno = raw_data_startno, endno = raw_data_endno, outfile = mean_rawdata_filename )


## Loads in raw mean file produced and manually applies dark subtraction and flatfield ##
conf = configparser.ConfigParser()
_ = conf.read(config_file)
calib_outpath  = conf['CALIB']['calib_outpath']
reduce_outpath = conf['REDUCTION']['reduce_outpath']
raw_staring_frame = fits.getdata( os.path.join(reduce_outpath,mean_rawdata_filename), 0 )
mean_dark_frame   = fits.getdata( os.path.join( calib_outpath,mean_dark_filename   ), 0 )
bpmask_frame      = fits.getdata( os.path.join( calib_outpath,bp_filename          ), 0 )
flatfield_frame   = fits.getdata( os.path.join( calib_outpath,flatfield_filename   ), 0 )
red_staring_frame = ( raw_staring_frame - mean_dark_frame ) / flatfield_frame
red_staring_frame = np.ma.masked_array( red_staring_frame, mask = bpmask_frame )
red_staring_frame = red_staring_frame.filled( np.nan )
hdu = fits.PrimaryHDU( red_staring_frame )
hdu.header['FILETYPE'] =   'Staring Reduct'
hdu.header['NFRAMES' ] = ( raw_data_endno-raw_data_startno , 'Number raw flat frames used'         )
hdu.header['STARTNO' ] = ( raw_data_startno                , 'First file number of obs data range' )
hdu.header['ENDNO'   ] = ( raw_data_endno                  , 'Last file number of obs data range'  )
hdu.header['DARKFILE'] = ( mean_dark_filename              , 'Dark file used'                      )
hdu.header['MASKFILE'] = ( bp_filename                     , 'Bad Pixel Mask file used'            )
hdu.header['FLFDFILE'] = ( flatfield_filename              , 'Flatfield file used'                 )
hdu.writeto( os.path.join( reduce_outpath, staring_data_filename ) )

# Plots reduced data 
fig, ax = P.subplots(figsize = (9,7))
im = ax.imshow( red_staring_frame, origin='lower' )
fig.colorbar(im, ax=ax)

# Use this section to zoom into where your source is in the image
# xlim = [ 400, 600 ]
# ylim = [ 400, 600 ]
# ax.set_xlim( xlim[0], xlim[1] )
# ax.set_ylim( ylim[0], ylim[1] )